<a href="https://colab.research.google.com/github/CelikAbdullah/deep-learning-notebooks/blob/main/Computer%20Vision/models/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [35]:
from tensorflow import keras

# ResNet

## ResNet34

### The stem component

In [ ]:
def stem(inputs):
  '''
  Build the stem component of ResNet34.
  Perform the initial coarse-level feature extraction.

  inputs: the input vector
  '''
  # a conv layer that outputs 64 feature maps
  x = keras.layers.Conv2D(filters=64,
                          kernel_size=7,
                          strides=2,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal",
                          )(inputs)

  # a max pooling layer that pools the 64 feature maps
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2, padding="same")(x)

  # following line would have the same effect as the previous max pooling layer
  #x = keras.layers.MaxPooling2D(pool_size=2)(x)


  return x

### The learner component

#### Residual block

A block that consists of two convolutional layers with the same number of filters for feature extraction. The input to the residual block will be added to the output of the last conv layer via an identity link.

In [ ]:
def residual_block(x, filters):
  '''
  A residual block with an identity link and
  two convolutional layers. All convolutional
  layers have the same number of filters.

  x       :  input into to the residual block
  filters :  nr. of filters of each conv layer
  '''

  # identity link, also referred to as "shortcut"
  shortcut = x

  # apply non-strided 3x3 convolutions for feature extraction
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=1,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal"
                          )(x)

  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=1,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal"
                          )(x)

  # add the output feature maps to the input
  # note: padding="same" preserved the size of the width & height
  #       so that adding input and output feature maps can work
  x = keras.layers.Add()([shortcut,x])

  # return the addition of the input & output feature maps
  return x


#### Conv block

A block that also consists of two convolutional layers. It will be placed after the last residual block of a residual block. It is responsible for doubling the number of output feature maps for the subsequent residual group.

In [ ]:
def conv_block(x, filters):
  '''
  A conv block with two strided convolutional layers that extract
  features, doubles the number of feature maps for the
  next residual group and reduces the size of the feature
  maps.

  x       : input to the conv block
  filters : number of filters
  '''
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=2 ,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal")(x)
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=2,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal")(x)

  # return the output feature maps
  return x

#### Residual group

Now, we can create the residual group. The learner component of ResNet34 has 4 residual groups.

In [ ]:
def residual_group(x, filters, blocks, conv=True):
  '''
  Create a residual group.

  x       : input to the residual group
  filters : number of filters for the conv layers within the residual blocks
  blocks  : number of residual blocks in the residual group
  conv    : flag that decides whether we need a conv block or not
            (note: all residual groups have one, except the last one.)
  '''

  # create the residual blocks of the residual group
  for _ in range(blocks):
    x = residual_block(x, filters)


  # create the conv block to double the number of feature maps for the
  # upcoming residual group and also reduce the size of the feature maps
  # via a stride of 2
  # (Note: Doubling of the number of feature maps has to be done since each residual
  # group doubles the number of filters in their residual blocks. If we don't
  # do this then the addition of feature maps at the end of a residual block
  # wouldn't work.
  if conv:
    x = conv_block(x, filters * 2)


  # return the output feature maps
  return x

Finally, we can put all pieces together to build the learner component of ResNet34:

In [ ]:
def learner(x):
  '''
  Build the learner component of ResNet34.
  Perform the detailed feature extraction and representational learning
  from the extracted coarse features.

  x: input to the learner
  '''

  # create the 1st residual group with 3 residual blocks.
  # Each residual block consists of 2 convolutional layers with 64 filters.
  x = residual_group(x, 64, 3)

  # create the 1st residual group with 3 residual blocks.
  # Each residual block consists of 2 convolutional layers with 128 filters.
  x = residual_group(x, 128, 3)

  # create the 1st residual group with 5 residual blocks.
  # Each residual block consists of 2 convolutional layers with 256 filters.
  x = residual_group(x, 256, 5)

  # create the 1st residual group with 2 residual blocks.
  # Each residual block consists of 2 convolutional layers with 512 filters.
  x = residual_group(x, 512, 2, False)

  # return the final output feature maps (latent space)
  return x

### The task component

In [ ]:
def task(x, classes):
  '''
  Build the task component that perform classification.

  x       : input to the task component
  classes : number of classes
  '''

  # global average pooling to reduce and flatten the
  # feature maps into a 1D vector
  x = keras.layers.GlobalAveragePooling2D()(x)

  # softmax layer
  outputs = keras.layers.Dense(units=classes,
                               activation="softmax",
                               kernel_initializer="he_normal")(x)


  # return the probability of each class
  return outputs

In [ ]:
inputs = keras.Input(shape=(224,224, 3))

In [ ]:
# perform the coarse-level feature extraction
coarse_features = stem(inputs)

# perform the detailed feature extraction & representational learning from the coarse features
detailed_features = learner(coarse_features)

# the task component performing classification
outputs = task(detailed_features, 1000)


model = keras.Model(inputs=inputs, outputs=outputs, name="resnet_34")
model.summary()

Model: "resnet_34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_34 (Conv2D)             (None, 112, 112, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 56, 56, 64)  0           ['conv2d_34[0][0]']              
                                                                                          

## ResNet v1

### The stem component

In [11]:
def stem(inputs):
  '''
  Build the stem component of ResNet v1

  inputs: input to the stem
  '''

  # apply zero-padding to preserve the size of the inputs
  # note: instead of a ZeroPadding2D layer, we could also
  #       use the padding="same" argument in conv layer below
  x = keras.layers.ZeroPadding2D(padding=(3,3))(inputs)

  # a conv layer to extract the coarse-level features from inputs
  x = keras.layers.Conv2D(filters=64,
                          kernel_size=7,
                          strides=2,
                          padding="valid",
                          use_bias=False,
                          kernel_initializer="he_normal"
                          )(x)

  # apply batch normalization
  x = keras.layers.BatchNormalization()(x)

  # apply ReLU activation fct
  x = keras.layers.ReLU()(x)

  # again, apply zero-padding
  x = keras.layers.ZeroPadding2D(padding=(1,1))(x)

  # perform strided max-pooling with a 3x3 pooling window
  x = keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)

  # return the output feature maps
  return x

### The learner component

#### The identity block

In [3]:
def identity_block(input, filters):
  '''
  Build a bottleneck residual block with an identity link.

  input   : input to the identity block
  filters : nr. of filters to use in the conv layer of the block
  '''

  # create the identity link, also referred to as "shortcut"
  shortcut = input

  # a 1x1 bottleneck convolution for dimensionality reduction
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal",
                          )(input)
  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)
  # perform ReLU
  x = keras.layers.ReLU()(x)


  # now, create the conv layers used for the residual path

  # a 3x3 conv layer
  x = keras.layers.Conv2D(filters = filters,
                          kernel_size=3,
                          strides=1,
                          padding="same",
                          use_bias=False,
                          kernel_initializer="he_normal")(x)
  # perform batch-normalization
  x = keras.layers.BatchNormalization()(x)
  # perform ReLU
  x = keras.layers.ReLU()(x)


  # a 1x1 linear projection conv layer for expanding the dimensionality
  x = keras.layers.Conv2D(filters=filters*4,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal")(x)

  # perform batch-normalization
  x = keras.layers.BatchNormalization()(x)

  # add the identity link (input) to the output of the residual path
  x = keras.layers.Add()([shortcut, x])

  # perform ReLU
  x = keras.layers.ReLU()(x)

  return x


#### The projection block

In [15]:
def projection_block(input, filters, strides=2):
  '''
  Build the bottleneck residual block with an identity link that uses
  a linear projection.

  Expand the number of filters.

  input : input to the block
  filters: nr. of filters in the conv layers of the block
  strides : stride value
  '''

  # create the identity link with linear projection
  shortcut = keras.layers.Conv2D(filters = filters * 4,
                                 kernel_size=1,
                                 strides=strides,
                                 use_bias=False,
                                 kernel_initializer="he_normal")(input)

  # apply batch normalization
  shortcut = keras.layers.BatchNormalization()(shortcut)


  # now, create the conv layers used for the residual path

  # 1.)
  # a 1x1 bottleneck conv layer for reducing the dimensionality
  # and applies feature pooling if strides=2
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=1,
                          strides=strides,
                          use_bias=False,
                          kernel_initializer="he_normal")(input)
  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)

  # apply ReLU
  x = keras.layers.ReLU()(x)

  # 2.)
  # a 3x3 conv layer acting as a bottleneck layer
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=1,
                          padding="same",
                          use_bias=False,
                          kernel_initializer="he_normal")(x)

  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)

  # apply ReLU
  x = keras.layers.ReLU()(x)

  # 3.)
  # a 1x1 linear projection conv layer for dimensionality restoration
  # recall that the nr. of input feature maps were increased on the identity link
  # so, we have to account for that when we want to add them to the output of
  # the residual path
  x = keras.layers.Conv2D(filters=filters*4,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal")(x)

  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)

  # finally, we can add the identity link with linear projection to
  # the output of the residual path
  x = keras.layers.Add()([shortcut, x])

  # apply ReLU
  x = keras.layers.ReLU()(x)

  # return the output feature maps
  return x

#### The residual group

In [5]:
def residual_group(input, filters, blocks, strides=2):
  '''
  Build a residual group.

  input   : input to the residual group
  filters : nr. of filters for the conv layers in the residual blocks
  blocks  : nr. of residual blocks in the residual group
  strides : stride value
  '''

  # each residual group starts with a projection block
  x = projection_block(input, filters, strides=strides)

  # followed by a couple of identity blocks
  for _ in range(blocks):
    x = identity_block(x, filters)

  # return the output feature maps
  return x

#### The learner

In [6]:
def learner(input, groups_spec):
  '''
  Build the learner component of ResNet v1.

  input   : input to the learner component
  groups  : a list of specifications (nr. of filters & blocks) for each group
  '''

  # the first group is not strided; so, we will use strides=1
  filters, blocks = groups_spec.pop(0)
  x = residual_group(input, filters, blocks, strides=1)

  # all other residual groups are strided
  for filters, blocks in groups_spec:
    x = residual_group(x, filters, blocks)


  # return the output feature maps
  return x

### The task component

In [17]:
def task(input, classes):
  '''
  Build the task component of ResNet v1.

  input   : input to the task component
  classes : nr. of classes
  '''

  # global average pooling to reduce and flatten the
  # feature maps into a 1D vector
  x = keras.layers.GlobalAveragePooling2D()(input)

  # create the softmax layer to output a probability for each class
  outputs = keras.layers.Dense(units=classes,
                               activation="softmax",
                               kernel_initializer="he_normal")(x)

  # return the output feature maps
  return outputs

### Build the ResNet v1 model

In [19]:
# create the specifications (nr. filters and blocks) for each group
groups = { 50 : [ (64, 3), (128, 4), (256, 6),  (512, 3) ],		# ResNet50
           101: [ (64, 3), (128, 4), (256, 23), (512, 3) ],		# ResNet101
           152: [ (64, 3), (128, 8), (256, 36), (512, 3) ]		# ResNet152
         }

# create the input tensor with a shape of 224x224x3
inputs = keras.Input(shape=(224,224,3))

# perform the coarse-level feature extraction
coarse_features = stem(inputs)

# perform the detailed feature extraction & representational learning from the coarse features
detailed_features = learner(coarse_features, groups[50])

# the task component performing classification
outputs = task(detailed_features, 1000)

# now, instantiate the ResNet50 model
model = keras.Model(inputs=inputs, outputs=outputs, name="ResNet50")
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_11 (ZeroPadding  (None, 230, 230, 3)  0          ['input_7[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv2d_137 (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_11[0][0]']      
                                )                                                          

## ResNet v1.5

### The stem component

In [28]:
def stem(inputs):
  '''
  Build the stem component of ResNet v1.5

  inputs: input to the stem
  '''

  # apply zero-padding to preserve the size of the inputs
  # note: instead of a ZeroPadding2D layer, we could also
  #       use the padding="same" argument in conv layer below
  x = keras.layers.ZeroPadding2D(padding=(3,3))(inputs)

  # a conv layer to extract the coarse-level features from inputs
  x = keras.layers.Conv2D(filters=64,
                          kernel_size=7,
                          strides=2,
                          padding="valid",
                          use_bias=False,
                          kernel_initializer="he_normal"
                          )(x)

  # apply batch normalization
  x = keras.layers.BatchNormalization()(x)

  # apply ReLU activation fct
  x = keras.layers.ReLU()(x)

  # again, apply zero-padding
  x = keras.layers.ZeroPadding2D(padding=(1,1))(x)

  # perform strided max-pooling with a 3x3 pooling window
  x = keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)

  # return the output feature maps
  return x

### The learner component

#### The identity block

In [29]:
def identity_block(input, filters):
  '''
  Build a bottleneck residual block with an identity link.

  input   : input to the identity block
  filters : nr. of filters to use in the conv layer of the block
  '''

  # create the identity link, also referred to as "shortcut"
  shortcut = input

  # a 1x1 bottleneck convolution for dimensionality reduction
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal",
                          )(input)
  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)
  # perform ReLU
  x = keras.layers.ReLU()(x)


  # now, create the conv layers used for the residual path

  # a 3x3 conv layer
  x = keras.layers.Conv2D(filters = filters,
                          kernel_size=3,
                          strides=1,
                          padding="same",
                          use_bias=False,
                          kernel_initializer="he_normal")(x)
  # perform batch-normalization
  x = keras.layers.BatchNormalization()(x)
  # perform ReLU
  x = keras.layers.ReLU()(x)


  # a 1x1 linear projection conv layer for expanding the dimensionality
  x = keras.layers.Conv2D(filters=filters*4,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal")(x)

  # perform batch-normalization
  x = keras.layers.BatchNormalization()(x)

  # add the identity link (input) to the output of the residual path
  x = keras.layers.Add()([shortcut, x])

  # perform ReLU
  x = keras.layers.ReLU()(x)

  return x


#### The projection block

In [30]:
def projection_block(input, filters, strides=2):
  '''
  Build the bottleneck residual block with an identity link that uses
  a linear projection.

  Expand the number of filters.

  input : input to the block
  filters: nr. of filters in the conv layers of the block
  strides : stride value
  '''

  # create the identity link with linear projection
  shortcut = keras.layers.Conv2D(filters = filters * 4,
                                 kernel_size=1,
                                 strides=strides,
                                 use_bias=False,
                                 kernel_initializer="he_normal")(input)

  # apply batch normalization
  shortcut = keras.layers.BatchNormalization()(shortcut)


  # now, create the conv layers used for the residual path

  # 1.)
  # a 1x1 bottleneck conv layer for reducing the dimensionality
  # and applies feature pooling if strides=2
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal")(input)
  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)

  # apply ReLU
  x = keras.layers.ReLU()(x)

  # 2.)
  # a 3x3 conv layer acting as a bottleneck layer
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=strides,
                          padding="same",
                          use_bias=False,
                          kernel_initializer="he_normal")(x)

  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)

  # apply ReLU
  x = keras.layers.ReLU()(x)

  # 3.)
  # a 1x1 linear projection conv layer for dimensionality restoration
  # recall that the nr. of input feature maps were increased on the identity link
  # so, we have to account for that when we want to add them to the output of
  # the residual path
  x = keras.layers.Conv2D(filters=filters*4,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal")(x)

  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)

  # finally, we can add the identity link with linear projection to
  # the output of the residual path
  x = keras.layers.Add()([shortcut, x])

  # apply ReLU
  x = keras.layers.ReLU()(x)

  # return the output feature maps
  return x

#### The residual group

In [31]:
def residual_group(input, filters, blocks, strides=2):
  '''
  Build a residual group.

  input   : input to the residual group
  filters : nr. of filters for the conv layers in the residual blocks
  blocks  : nr. of residual blocks in the residual group
  strides : stride value
  '''

  # each residual group starts with a projection block
  x = projection_block(input, filters, strides=strides)

  # followed by a couple of identity blocks
  for _ in range(blocks):
    x = identity_block(x, filters)

  # return the output feature maps
  return x

#### The learner

In [32]:
def learner(input, groups_spec):
  '''
  Build the learner component of ResNet v1.5.

  input   : input to the learner component
  groups  : a list of specifications (nr. of filters & blocks) for each group
  '''

  # the first group is not strided; so, we will use strides=1
  filters, blocks = groups_spec.pop(0)
  x = residual_group(input, filters, blocks, strides=1)

  # all other residual groups are strided
  for filters, blocks in groups_spec:
    x = residual_group(x, filters, blocks)


  # return the output feature maps
  return x

### The task component

In [33]:
def task(input, classes):
  '''
  Build the task component of ResNet v1.5.

  input   : input to the task component
  classes : nr. of classes
  '''

  # global average pooling to reduce and flatten the
  # feature maps into a 1D vector
  x = keras.layers.GlobalAveragePooling2D()(input)

  # create the softmax layer to output a probability for each class
  outputs = keras.layers.Dense(units=classes,
                               activation="softmax",
                               kernel_initializer="he_normal")(x)

  # return the output feature maps
  return outputs

### Build the ResNet v1.5 model

In [34]:
# create the specifications (nr. filters and blocks) for each group
groups = { 50 : [ (64, 3), (128, 4), (256, 6),  (512, 3) ],		# ResNet50
           101: [ (64, 3), (128, 4), (256, 23), (512, 3) ],		# ResNet101
           152: [ (64, 3), (128, 8), (256, 36), (512, 3) ]		# ResNet152
         }

# create the input tensor with a shape of 224x224x3
inputs = keras.Input(shape=(224,224,3))

# perform the coarse-level feature extraction
coarse_features = stem(inputs)

# perform the detailed feature extraction & representational learning from the coarse features
detailed_features = learner(coarse_features, groups[50])

# the task component performing classification
outputs = task(detailed_features, 1000)

# now, instantiate the ResNet50 model
model = keras.Model(inputs=inputs, outputs=outputs, name="ResNet50")
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_15 (ZeroPadding  (None, 230, 230, 3)  0          ['input_9[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv2d_267 (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_15[0][0]']      
                                )                                                          

## ResNet v2

### The stem component

In [36]:
def stem(inputs):
  '''
  Build the stem component of ResNet v1.5

  inputs: input to the stem
  '''

  # apply zero-padding to preserve the size of the inputs
  # note: instead of a ZeroPadding2D layer, we could also
  #       use the padding="same" argument in conv layer below
  x = keras.layers.ZeroPadding2D(padding=(3,3))(inputs)

  # a conv layer to extract the coarse-level features from inputs
  x = keras.layers.Conv2D(filters=64,
                          kernel_size=7,
                          strides=2,
                          padding="valid",
                          use_bias=False,
                          kernel_initializer="he_normal"
                          )(x)

  # apply batch normalization
  x = keras.layers.BatchNormalization()(x)

  # apply ReLU activation fct
  x = keras.layers.ReLU()(x)

  # again, apply zero-padding
  x = keras.layers.ZeroPadding2D(padding=(1,1))(x)

  # perform strided max-pooling with a 3x3 pooling window
  x = keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)

  # return the output feature maps
  return x

### The learner component

#### The identity block

In [37]:
def identity_block(input, filters):
  '''
  Build a bottleneck residual block with an identity link.

  input   : input to the identity block
  filters : nr. of filters to use in the conv layer of the block
  '''

  # create the identity link, also referred to as "shortcut"
  shortcut = input

  # perform batch normalization
  x = keras.layers.BatchNormalization()(input)

  # perform ReLU
  x = keras.layers.ReLU()(x)

  # a 1x1 bottleneck convolution for dimensionality reduction
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal",
                          )(x)



  # now, create the conv layers used for the residual path

  # perform batch-normalization
  x = keras.layers.BatchNormalization()(x)

  # perform ReLU
  x = keras.layers.ReLU()(x)

  # a 3x3 conv layer
  x = keras.layers.Conv2D(filters = filters,
                          kernel_size=3,
                          strides=1,
                          padding="same",
                          use_bias=False,
                          kernel_initializer="he_normal")(x)

  # perform batch-normalization
  x = keras.layers.BatchNormalization()(x)

  # perform ReLU
  x = keras.layers.ReLU()(x)

  # a 1x1 linear projection conv layer for expanding the dimensionality
  x = keras.layers.Conv2D(filters=filters*4,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal")(x)



  # add the identity link (input) to the output of the residual path
  x = keras.layers.Add()([shortcut, x])


  return x

#### The projection block

In [38]:
def projection_block(input, filters, strides=2):
  '''
  Build the bottleneck residual block with an identity link that uses
  a linear projection.

  Expand the number of filters.

  input : input to the block
  filters: nr. of filters in the conv layers of the block
  strides : stride value
  '''

  # apply batch normalization
  shortcut = keras.layers.BatchNormalization()(input)

  # create the identity link with linear projection
  shortcut = keras.layers.Conv2D(filters = filters * 4,
                                 kernel_size=1,
                                 strides=strides,
                                 use_bias=False,
                                 kernel_initializer="he_normal")(shortcut)




  # now, create the conv layers used for the residual path

  # perform batch normalization
  x = keras.layers.BatchNormalization()(input)

  # apply ReLU
  x = keras.layers.ReLU()(x)

  # 1.)
  # a 1x1 bottleneck conv layer for reducing the dimensionality
  # and applies feature pooling if strides=2
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal")(x)

  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)

  # apply ReLU
  x = keras.layers.ReLU()(x)

  # 2.)
  # a 3x3 conv layer acting as a bottleneck layer
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=strides,
                          padding="same",
                          use_bias=False,
                          kernel_initializer="he_normal")(x)


  # perform batch normalization
  x = keras.layers.BatchNormalization()(x)

  # apply ReLU
  x = keras.layers.ReLU()(x)

  # 3.)
  # a 1x1 linear projection conv layer for dimensionality restoration
  # recall that the nr. of input feature maps were increased on the identity link
  # so, we have to account for that when we want to add them to the output of
  # the residual path
  x = keras.layers.Conv2D(filters=filters*4,
                          kernel_size=1,
                          strides=1,
                          use_bias=False,
                          kernel_initializer="he_normal")(x)



  # finally, we can add the identity link with linear projection to
  # the output of the residual path
  x = keras.layers.Add()([shortcut, x])

  # return the output feature maps
  return x

#### The residual group

In [39]:
def residual_group(input, filters, blocks, strides=2):
  '''
  Build a residual group.

  input   : input to the residual group
  filters : nr. of filters for the conv layers in the residual blocks
  blocks  : nr. of residual blocks in the residual group
  strides : stride value
  '''

  # each residual group starts with a projection block
  x = projection_block(input, filters, strides=strides)

  # followed by a couple of identity blocks
  for _ in range(blocks):
    x = identity_block(x, filters)

  # return the output feature maps
  return x

#### The learner

In [40]:
def learner(input, groups_spec):
  '''
  Build the learner component of ResNet v2.

  input   : input to the learner component
  groups  : a list of specifications (nr. of filters & blocks) for each group
  '''

  # the first group is not strided; so, we will use strides=1
  filters, blocks = groups_spec.pop(0)
  x = residual_group(input, filters, blocks, strides=1)

  # all other residual groups are strided
  for filters, blocks in groups_spec:
    x = residual_group(x, filters, blocks)


  # return the output feature maps
  return x

### The task component

In [41]:
def task(input, classes):
  '''
  Build the task component of ResNet v2.

  input   : input to the task component
  classes : nr. of classes
  '''

  # global average pooling to reduce and flatten the
  # feature maps into a 1D vector
  x = keras.layers.GlobalAveragePooling2D()(input)

  # create the softmax layer to output a probability for each class
  outputs = keras.layers.Dense(units=classes,
                               activation="softmax",
                               kernel_initializer="he_normal")(x)

  # return the output feature maps
  return outputs

### Build the ResNet v2 model

In [42]:
# create the specifications (nr. filters and blocks) for each group
groups = { 50 : [ (64, 3), (128, 4), (256, 6),  (512, 3) ],		# ResNet50
           101: [ (64, 3), (128, 4), (256, 23), (512, 3) ],		# ResNet101
           152: [ (64, 3), (128, 8), (256, 36), (512, 3) ]		# ResNet152
         }

# create the input tensor with a shape of 224x224x3
inputs = keras.Input(shape=(224,224,3))

# perform the coarse-level feature extraction
coarse_features = stem(inputs)

# perform the detailed feature extraction & representational learning from the coarse features
detailed_features = learner(coarse_features, groups[50])

# the task component performing classification
outputs = task(detailed_features, 1000)

# now, instantiate the ResNet50 model
model = keras.Model(inputs=inputs, outputs=outputs, name="ResNet50")
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_17 (ZeroPadding  (None, 230, 230, 3)  0          ['input_10[0][0]']               
 2D)                                                                                              
                                                                                                  
 conv2d_332 (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_17[0][0]']      
                                )                                                          